Initial setup, imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import gymnasium as gym
import random
from queue import Queue

In [2]:
#create a basic grid map
map = tf.zeros([16, 16], tf.int32)


In [3]:
column_indexes = [0, 15]
row_indexes = [0, 15]

column_indices=[]
row_indices=[]

for idx in column_indexes:
    column_indices.extend([[i, idx] for i in range(16)])
for idx in row_indexes:
    row_indices.extend([[idx, i] for i in range(16)])

# Combine row and column indices
indices = tf.constant(column_indices + row_indices, dtype=tf.int32)
print(indices)
# Create the corresponding updates
updates = tf.ones((len(indices),), dtype=tf.int32)
print(updates)
map = tf.tensor_scatter_nd_update(
    map,
    indices = indices,
    updates = updates
)

print(map)

tf.Tensor(
[[ 0  0]
 [ 1  0]
 [ 2  0]
 [ 3  0]
 [ 4  0]
 [ 5  0]
 [ 6  0]
 [ 7  0]
 [ 8  0]
 [ 9  0]
 [10  0]
 [11  0]
 [12  0]
 [13  0]
 [14  0]
 [15  0]
 [ 0 15]
 [ 1 15]
 [ 2 15]
 [ 3 15]
 [ 4 15]
 [ 5 15]
 [ 6 15]
 [ 7 15]
 [ 8 15]
 [ 9 15]
 [10 15]
 [11 15]
 [12 15]
 [13 15]
 [14 15]
 [15 15]
 [ 0  0]
 [ 0  1]
 [ 0  2]
 [ 0  3]
 [ 0  4]
 [ 0  5]
 [ 0  6]
 [ 0  7]
 [ 0  8]
 [ 0  9]
 [ 0 10]
 [ 0 11]
 [ 0 12]
 [ 0 13]
 [ 0 14]
 [ 0 15]
 [15  0]
 [15  1]
 [15  2]
 [15  3]
 [15  4]
 [15  5]
 [15  6]
 [15  7]
 [15  8]
 [15  9]
 [15 10]
 [15 11]
 [15 12]
 [15 13]
 [15 14]
 [15 15]], shape=(64, 2), dtype=int32)
tf.Tensor(
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], shape=(64,), dtype=int32)
tf.Tensor(
[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 

In [4]:
npmap = np.zeros((16, 16), np.int8)
idxs = [0, 15]
for i in idxs:
    npmap[i, :] = 1
    npmap[:, i] = 1



In [15]:
class Node: #nodes feature coordinate system, wall boolean and neighbour list
    def __init__(self, row, col):
        self.row = row
        self.col = col
        self.neighbours=[]
        self.is_wall = True

class Map: #map containing grid of nodes, square based on map_size
    def __init__(self, map_size):
        self.grid = [[Node(row, col) for col in range(map_size)] for row in range(map_size)]
        self.size = map_size

class UnionFind: #unionFind algorithm for joining nodes
    def __init__(self, map):
        self.parent=[[(r, c) for c in range(map.size)] for r in range(map.size)] #parents are coordinates as id
        self.rank=[[0 for c in range(map.size)] for r in range(map.size)] #rank for union by rank
        
    def find(self, coord): #check parent of cnode by coordinate
        r, c = coord
        if self.parent[r][c] != coord: #if parent is not itself
            self.parent[r][c] = self.find(self.parent[r][c]) #find actual parent
        return self.parent[r][c] #return parent
    
    def union(self, coord1, coord2): #join two nodes
        print("UNION: coord1:", coord1, "coord2:", coord2)
        root1 = self.find(coord1) #get parent of coord1
        root2 = self.find(coord2) #get parent of coord2

        print("UNION: root1:", root1, "root2:", root2)
        if root1 != root2: #
            r1, c1 = root1
            r2, c2 = root2
            print("RANKS: rank1 =", self.rank[r1][c1], ", rank2 =", self.rank[r2][c2])
            
            if self.rank[r1][c1] > self.rank[r2][c2]:
                self.parent[r2][c2] = root1
            elif self.rank[r1][c1] < self.rank[r2][c2]:
                self.parent[r1][c1] = root2
            else:
                self.parent[r2][c2] = root1
                self.rank[r1][c1] += 1

def createBareMap(size):
    map = Map(size)
    #iterate through map rows and cols
    for row in range(map.size):
        for col in range(map.size):
            node = map.grid[row][col]
            if row > 0:
                node.neighbours.append(map.grid[row - 1][col]) #north neighbour
            if row < map.size - 1:
                node.neighbours.append(map.grid[row + 1][col])  #south neighbour
            if col > 0:
                node.neighbours.append(map.grid[row][col - 1])  #west neighbour
            if col < map.size - 1:
                node.neighbours.append(map.grid[row][col + 1])  #east neighbour

    #set initial path cells
    for row in range(map.size):
        for col in range(map.size):
            if row % 2 == 1 and col % 2 == 1:
                map.grid[row][col].is_wall = False
    return map

def drawMap(map):
    for row in range(map.size):
            print(''.join(' ' if not node.is_wall else '#' for node in map.grid[row]))

def getEdges(map): #if first pass get every 2nd wall, allowing for perfect maze
    edges=[]
    for row in range(1, map.size, 2): #start at 1, step by two to only get path cells
        for col in range(1, map.size, 2):
            if row + 2 < map.size:  #vert wall
                wall = map.grid[row + 1][col]
                cell1 = map.grid[row][col]
                cell2 = map.grid[row + 2][col]
                edges.append((wall, cell1, cell2))  # (wall, cell1, cell2)
            if col + 2 < map.size:  #horizontal wall
                wall = map.grid[row][col + 1]
                cell1 = map.grid[row][col]
                cell2 = map.grid[row][col + 2]
                edges.append((wall, cell1, cell2))
    return edges

def getRemainingWalls(map):
    walls=[]
    for row in range(1, map.size - 1): #start at 1, step by two to only get path cells
            for col in range(1, map.size - 1):
                if map.grid[row][col].is_wall:  #vert wall
                    walls.append(map.grid[row][col])  # (wall, cell1, cell2)
    return walls
        
    
def removeWalls(uf, edges): #takes a unionfind and edge list
    for wall, cell1, cell2 in edges:
        print(" ")
        print("debug log - process wall:", (wall.row, wall.col))
        coord1 = (cell1.row, cell1.col)
        coord2 = (cell2.row, cell2.col)
        if uf.find(coord1) != uf.find(coord2):
            uf.union(coord1, coord2)
            wall.is_wall = False
            print("wall removed: ", (wall.row, wall.col))
        else:
            print("skipping unneccessary wall removal:", (wall.row, wall.col))

def isBoundary(node, map):
    if node.row == 0:
        return True
    elif node.row == map.size - 1:
        return True
    elif node.col == 0:
        return True
    elif node.col == map.size - 1:
        return True
    else:
        return False

def createCycles(cycle_bias, cluster_strength, cluster_type, map):
    num_edges = 0
    remaining_walls = getRemainingWalls(map)
    num_cycles = np.floor(cycle_bias * len(remaining_walls))

    print("Cycle bias, num edges, num cycles: ", (cycle_bias, num_edges, num_cycles))

    random.shuffle(remaining_walls)
    print("map before cycles")
    drawMap(map)
    counter = 0

    for wall in remaining_walls: #get a wall and remove it
        if counter < num_cycles:
            wall.is_wall = False
            counter += 1

            match cluster_type: #cluster removal of walls based on input, centred on removed wall
                case "pocket":
                    nearest=[]
                    for row in range(wall.row - 2, wall.row + 3):
                        for col in range(wall.col - 2, wall.col + 3):
                            if row < map.size and row > 0 and col < map.size and col > 0:                            
                                if map.grid[row][col].is_wall:
                                    nearest.append(map.grid[row][col])
                    for close_wall in nearest:
                        if random.random() < cluster_strength:
                            close_wall.is_wall = False
                            counter += 1

                case "radial": #flood fill search across map for walls
                    print("debug: into radial")
                    visited = {(wall.row, wall.col)}
                    #todo maybe make a list of queued?
                    q = Queue()
                    for neighbour in wall.neighbours:
                        q.put(neighbour)
                        print("neighbour being added: ", neighbour)

                    #iterate through neighbours and add new neighbours
                    while q.empty() != True and counter < num_cycles:
                        next = q.get()
                        if next in visited:
                            print("next is visited: ", (next.row, next.col))
                        #print("next: ", (next.row, next.col))
                        for next_neighbour in next.neighbours: #add new neighbours if not visted before
                            if (next_neighbour.row, next_neighbour.col) not in visited and not isBoundary(next_neighbour, map):
                                #print("debug: neighbour: ", (next_neighbour.row, next_neighbour.col))
                                #print("debug: visited: ", visited)
                                q.put(next_neighbour)

                        if next.is_wall == True and not isBoundary(next, map): #if current node is a wall, get coords
                            x1, y1 = wall.row, wall.col
                            x2, y2 = next.row, next.col
                            if random.random() < cluster_strength / (abs(x1 - x2) + abs(y1 - y2)): #manhattan distance reduction of removal chance
                                next.is_wall = False #remove wall if chance succeeds
                                counter += 1

                        visited.add((next.row, next.col))

                        

                case _:
                    #no clustering needed by default
                    continue
        else:
            break
    
    print("map after cycles")
    drawMap(map)


In [16]:
#init the map with size
map = createBareMap(15)

#creating list of edges for kruskal
edges = getEdges(map)

drawMap(map)
print(" ")

#shuffle edges
random.shuffle(edges)

uf = UnionFind(map)  #start unionfind with a map
removeWalls(uf, edges)

createCycles(0.5, 0.5, "radial",  map)

###############
# # # # # # # #
###############
# # # # # # # #
###############
# # # # # # # #
###############
# # # # # # # #
###############
# # # # # # # #
###############
# # # # # # # #
###############
# # # # # # # #
###############
 
 
debug log - process wall: (10, 9)
UNION: coord1: (9, 9) coord2: (11, 9)
UNION: root1: (9, 9) root2: (11, 9)
RANKS: rank1 = 0 , rank2 = 0
wall removed:  (10, 9)
 
debug log - process wall: (9, 6)
UNION: coord1: (9, 5) coord2: (9, 7)
UNION: root1: (9, 5) root2: (9, 7)
RANKS: rank1 = 0 , rank2 = 0
wall removed:  (9, 6)
 
debug log - process wall: (9, 4)
UNION: coord1: (9, 3) coord2: (9, 5)
UNION: root1: (9, 3) root2: (9, 5)
RANKS: rank1 = 0 , rank2 = 1
wall removed:  (9, 4)
 
debug log - process wall: (1, 10)
UNION: coord1: (1, 9) coord2: (1, 11)
UNION: root1: (1, 9) root2: (1, 11)
RANKS: rank1 = 0 , rank2 = 0
wall removed:  (1, 10)
 
debug log - process wall: (11, 6)
UNION: coord1: (11, 5) coord2: (11, 7)
UNION: root1: (11, 5) root2: (11, 7)
RANKS:

In [18]:

drawMap(map)

###############
#             #
# #   # # # # #
#   # #   #   #
#     ###   # #
#             #
# #       # # #
# #         # #
# # #       ###
#             #
######      ###
#           # #
#######     # #
#             #
###############
